In [64]:
from bs4 import BeautifulSoup
import requests
import csv

In [66]:
# Connect to website

# Amazon search results page for "data analyst tshirt"
URL = 'https://www.amazon.com/s?k=data%2Banalyst%2Btshirt&crid=M7SWQ9FKTZG9&qid=1740884136&sprefix=data%2Banalyst%2Btshirt%2Caps%2C121&xpid=8-6USGCdPy3cd&ref=sr_pg_1'

# Headers to mimic a real browser request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9,de;q=0.8",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Referer": "https://www.amazon.com/",
    "DNT": "1",
    "Connection": "keep-alive"}

# Send request to Amazon search page
page = requests.get(URL, headers=headers)

#Pulling in the page content
soup = BeautifulSoup(page.text, "html.parser")

print(soup)

<!DOCTYPE html>
<html class="a-no-js" data-19ax5a9jf="dingo" lang="en-us"><!-- sp:feature:head-start -->
<head><script>var aPageStart = (new Date()).getTime();</script><meta charset="utf-8"/>
<!-- sp:end-feature:head-start -->
<!-- sp:feature:csm:head-open-part1 -->
<script type="text/javascript">var ue_t0=ue_t0||+new Date();</script>
<!-- sp:end-feature:csm:head-open-part1 -->
<!-- sp:feature:cs-optimization -->
<meta content="on" http-equiv="x-dns-prefetch-control"/>
<link href="https://images-na.ssl-images-amazon.com" rel="dns-prefetch"/>
<link href="https://m.media-amazon.com" rel="dns-prefetch"/>
<link href="https://completion.amazon.com" rel="dns-prefetch"/>
<!-- sp:end-feature:cs-optimization -->
<!-- sp:feature:csm:head-open-part2 -->
<script type="text/javascript">
window.ue_ihb = (window.ue_ihb || window.ueinit || 0) + 1;
if (window.ue_ihb === 1) {

var ue_csm = window,
    ue_hob = +new Date();
(function(d){var e=d.ue=d.ue||{},f=Date.now||function(){return+new Date};e.d=func

In [72]:
# Find all product links on the search results page
product_links = []
for link in soup.find_all("a",class_="a-link-normal s-no-outline"):
    product_url = "https://www.amazon.com/" + link["href"]
    product_links.append(product_url)

# Open CSV file to store product data
with open('AmazonWebScrappingDataset.csv','w',newline='',encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(['Product Title','Price','Product Link']) # Header row

    # Loop through each product link and extract details
    for product_url in product_links[:100]:
        product_page = requests.get(product_url, headers=headers)
        product_soup = BeautifulSoup(product_page.text, "html.parser")

        # Check for Captcha before scraping
        if "Enter the characters you see below" in product_soup.text:
            print("Blocked by Captcha! Skipping:", product_url)
            continue
            
        # Extract product title
        title_element = product_soup.find(id = 'productTitle')
        title = title_element.get_text(strip=True) if title_element else "N/A"

        # Extract product price
        price_element = product_soup.find("span", class_="aok-offscreen")
        price = price_element.get_text(strip=True) if price_element else "N/A"

        # Save to CSV
        writer.writerow([title, price, product_url])
        
        print(f"Scraped: {title} - {price}")

print("Scraping completed. Data saved in 'AmazonWebScraperDataset.csv'.")

Scraped: Funny Data Analyst For Men Software Engineering Scientist T-Shirt - $21.99
Scraped: That Wasn't Very Data Driven of You Data Analyst Funny Cat T-Shirt - $17.99
Scraped: That Wasn't Very Data Driven of You Funny Data Analyst T-Shirt - $16.97
Scraped: Data Analyst Definition Data Scientist Data Science Expert T-Shirt - $19.99
Scraped: Got Data Funny Business Data Analyst T-Shirt - $15.99
Scraped: funny data analyst definition data scientist T-Shirt - $19.99
Scraped: That Wasn't Very Data Driven of You Funny Data Analyst Geek T-Shirt - $16.95
Scraped: Pi Day T Shirt Kids Mens Graphic Math School Days Novelty Teachers Funny T-Shirt - $14.99 with 12 percent savings
Scraped: Sarcasm Periodic Table Element Weird Science Joke Gift T-Shirt - $19.95
Scraped: Never Forget Mens Cassette Tape VHS Gamer Old School Mens Very Funny T Shirt - $14.99
Scraped: Star Wars Wrong Droids Funny Comic Disney+ T-Shirt - $22.99
Scraped: Star Wars Funny Darth Vader and Boba Fett Comic T-Shirt - $22.99
Scr

In [266]:
import pandas as pd

# Load the dataset
file_path = r"C:\Users\Lenovo\Amazon Web Scrapping Project\AmazonWebScrappingDataset.csv"
df_amazon = pd.read_csv(file_path)

In [268]:
df_amazon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Product Title  70 non-null     object
 1   Price          70 non-null     object
 2   Product Link   72 non-null     object
dtypes: object(3)
memory usage: 1.8+ KB


In [270]:
df_amazon.head(5)

,Product Title,Price,Product Link
0,Funny Data Analyst For Men Software Engineerin...,$21.99,https://www.amazon.com//sspa/click?ie=UTF8&spc...
1,That Wasn't Very Data Driven of You Data Analy...,$17.99,https://www.amazon.com//sspa/click?ie=UTF8&spc...
2,That Wasn't Very Data Driven of You Funny Data...,$16.97,https://www.amazon.com//sspa/click?ie=UTF8&spc...
3,Data Analyst Definition Data Scientist Data Sc...,$19.99,https://www.amazon.com//sspa/click?ie=UTF8&spc...
4,Got Data Funny Business Data Analyst T-Shirt,$15.99,https://www.amazon.com//Funny-Data-Systems-Bus...


In [272]:
#Converting the "Price" column from string to numeric and remove '$' sign and extra details from "Price" column.

import re

df_amazon['Price'] = df_amazon['Price'].apply(lambda x: float(re.search(r"\d+\.\d+", str(x)).group()) if re.search(r"\d+\.\d+", str(x)) else None)


In [274]:
#Handling missing values in the "Product Title" and "Price" columns.

df_amazon = df_amazon.dropna(subset=['Product Title', 'Price'])

In [276]:
#Computing basic price statistics (mean, median, min, max).

print(df_amazon['Price'].describe())

count    70.000000
mean     17.768286
std       2.664402
min      13.380000
25%      14.990000
50%      17.990000
75%      19.990000
max      24.990000
Name: Price, dtype: float64


In [278]:
#Top 5 Most Expensive T-Shirts
top_expensive = df_amazon.nlargest(5, 'Price')
print(top_expensive)

                                        Product Title  Price  \
34  Black Unisex Casual Modern Unique Original T-S...  24.99   
10  Star Wars Wrong Droids Funny Comic Disney+ T-S...  22.99   
11  Star Wars Funny Darth Vader and Boba Fett Comi...  22.99   
68  Star Wars Wrong Droids Funny Comic Disney+ T-S...  22.99   
0   Funny Data Analyst For Men Software Engineerin...  21.99   

                                         Product Link  
34  https://www.amazon.com//sspa/click?ie=UTF8&spc...  
10  https://www.amazon.com//sspa/click?ie=UTF8&spc...  
11  https://www.amazon.com//sspa/click?ie=UTF8&spc...  
68  https://www.amazon.com//sspa/click?ie=UTF8&spc...  
0   https://www.amazon.com//sspa/click?ie=UTF8&spc...  


In [280]:
#Top 5 Cheapest T-Shirts
top_cheapest = df_amazon.nsmallest(5, 'Price')
print(top_cheapest)

                                        Product Title  Price  \
15      Data Analyst Shirt - Torture The Data T-Shirt  13.38   
53  That Wasn't Very Data Driven Of You Funny Data...  13.38   
38  Data Statistics Math Computer Science Shirt An...  13.95   
27  That Wasn't Very Data Driven of You Funny Data...  13.99   
52                                            T-Shirt  13.99   

                                         Product Link  
15  https://www.amazon.com//Data-Analyst-Shirt-Tor...  
53  https://www.amazon.com//Data-Driven-Funny-Anal...  
38  https://www.amazon.com//Data-Statistics-Comput...  
27  https://www.amazon.com//Data-Driven-Analyst-Ge...  
52  https://www.amazon.com//Funny-Data-Systems-Bus...  


In [282]:
# Connect to website

# Ebay search results page for "data analyst tshirt"
URL = 'https://www.ebay.com/sch/i.html?_nkw=data%2Banalyst%2Btshirts&_sacat=0&_from=R40&_trksid=p4432023.m570.l1313'

# Headers to mimic a real browser request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9,de;q=0.8",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Referer": "https://www.ebay.com/",
    "DNT": "1",
    "Connection": "keep-alive"}

# Send request to Ebay search page
page = requests.get(URL, headers=headers)

#Pulling in the page content
soup = BeautifulSoup(page.text, "html.parser")

print(soup)

<!DOCTYPE html>
<!--[if IE 9]><html class="ie9" lang="en"><![endif]--><!--[if gt IE 9]><!--><html lang="en"><!--<![endif]--><!--M#s0-2--><noscript class="x-page-config"></noscript><!--M/--><head><!--M#s0-5--><meta content="IE=edge" http-equiv="X-UA-Compatible"/><script>
    (function () {
        'use strict';
        if (window.PerformanceObserver && performance && performance.mark && performance.getEntriesByName) {
            window.SRP = window.SRP || {};
            var paintObserver = new window.PerformanceObserver(function (list) {
                var paintEntries = list.getEntries();
                paintEntries.sort(function (a, b) {
                    return a.startTime - b.startTime;
                });
                // begin looking for TTI at first contentful paint
                if (!paintEntries || paintEntries.length < 2) {
                    // to avoid undefined issue on Safari
                    return;
                }
                var interactiveWindow = 

In [228]:
# Find all product links on the search results page
product_links = []
for item in soup.find_all("a", class_="s-item__link"):
    product_url = item["href"]
    product_links.append(product_url)

# Open CSV file to store product data
with open('EbayWebScrapingDataset.csv', 'w', newline='', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(['Product Title', 'Price', 'Product Link'])  # Header row

    # Loop through each product link and extract details
    for product_url in product_links[:100]:
        product_page = requests.get(product_url, headers=headers)
        product_soup = BeautifulSoup(product_page.text, "html.parser")
        
        # Extract product title
        title_element = product_soup.find("h1", class_="x-item-title__mainTitle")
        title = title_element.get_text(strip=True) if title_element else "N/A"

        # Extract product price
        price_element = product_soup.find("div", class_="x-price-primary")
        price = price_element.get_text(strip=True) if price_element else "N/A"

        # Save to CSV
        writer.writerow([title, price, product_url])
        
        print(f"Scraped: {title} - {price}")

print("Scraping completed. Data saved in 'EbayWebScrapingDataset.csv'.")

Scraped: N/A - N/A
Scraped: N/A - N/A
Scraped: That Wasn't Very Data Driven of You Funny Data Analyst Geek T-Shirt - US $14.99
Scraped: Funny Data Analyst That Wasn't Very Data Driven of You T-Shirt - US $14.99
Scraped: Funny Data Analyst That Wasn't Very Data Driven of You T-Shirt - US $14.99
Scraped: Data Analyst Statistic Scientist Funny Data Engineer Nerd T-Shirt - US $16.97
Scraped: Personalized Analyst Unisex T-shirt Custom name Business Data Analyst Sand Blue - US $22.49
Scraped: I Love My Data Analyst T shirt I Heart My Data Analyst Tee - US $16.95
Scraped: Data Analyst In Progress T-Shirt Funny Data Analyst Tee Data Science Shirt - US $20.61
Scraped: Limited For Data Analyst T-Shirt Made in the USA Size S to 5XL - US $21.78
Scraped: Data Analyst That Wasn't Very Data Driven of You T-shirt - US $6.98
Scraped: Data Analyst Unisex T-Shirt My First Word Was Data Tee Funny Data Analyst Shirt - US $20.61
Scraped: Data Analyst Softstyle T-Shirt - Data Driven Decision Focused - US $20

In [284]:
import pandas as pd

# Load the dataset
file_path = r"C:\Users\Lenovo\Amazon Web Scrapping Project\EbayWebScrapingDataset.csv"
df_ebay = pd.read_csv(file_path)

In [286]:
df_ebay.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Product Title  68 non-null     object
 1   Price          68 non-null     object
 2   Product Link   70 non-null     object
dtypes: object(3)
memory usage: 1.8+ KB


In [288]:
df_ebay.head(5)

,Product Title,Price,Product Link
0,NaN,NaN,https://ebay.com/itm/123456?itmmeta=012DEW30YG...
1,NaN,NaN,https://ebay.com/itm/123456?itmmeta=012DEW30YG...
2,That Wasn't Very Data Driven of You Funny Data...,US $14.99,https://www.ebay.com/itm/146063611132?_skw=dat...
3,Funny Data Analyst That Wasn't Very Data Drive...,US $14.99,https://www.ebay.com/itm/146113310188?_skw=dat...
4,Funny Data Analyst That Wasn't Very Data Drive...,US $14.99,https://www.ebay.com/itm/146095688507?_skw=dat...


In [290]:
#Converting the "Price" column from string to numeric and remove extra details from "Price" column.

import re

df_ebay['Price'] = df_ebay['Price'].apply(lambda x: float(re.search(r"\d+\.\d+", str(x)).group()) if re.search(r"\d+\.\d+", str(x)) else None)

In [292]:
#Handling missing values in the "Product Title" and "Price" columns.

df_ebay = df_ebay.dropna(subset=['Product Title', 'Price'])

In [294]:
#Computing basic price statistics (mean, median, min, max).

print(df_ebay['Price'].describe())

count    68.000000
mean     21.124412
std      12.806841
min       6.980000
25%      16.990000
50%      19.380000
75%      21.827500
max      99.950000
Name: Price, dtype: float64


In [296]:
df_amazon.to_csv("AmazonWebScrapingDataset_Cleaned.csv", index=False)

In [298]:
df_ebay.to_csv("EbayWebScrapingDataset_Cleaned.csv", index=False)

In [300]:
pip install mysql-connector-python


   ---------------------------------------- 0.0/16.1 MB ? eta -:--:--
   - -------------------------------------- 0.8/16.1 MB 6.7 MB/s eta 0:00:03
   ---- ----------------------------------- 1.8/16.1 MB 4.8 MB/s eta 0:00:03
   ------- -------------------------------- 3.1/16.1 MB 5.4 MB/s eta 0:00:03
   ----------- ---------------------------- 4.7/16.1 MB 6.1 MB/s eta 0:00:02
   --------------- ------------------------ 6.3/16.1 MB 6.2 MB/s eta 0:00:02
   ------------------- -------------------- 7.9/16.1 MB 6.4 MB/s eta 0:00:02
   ----------------------- ---------------- 9.4/16.1 MB 6.5 MB/s eta 0:00:02
   -------------------------- ------------- 10.7/16.1 MB 6.5 MB/s eta 0:00:01
   ------------------------------ --------- 12.3/16.1 MB 6.6 MB/s eta 0:00:01
   ----------------------------------- ---- 14.4/16.1 MB 6.9 MB/s eta 0:00:01
   -------------------------------------- - 15.5/16.1 MB 6.8 MB/s eta 0:00:01
   ---------------------------------------- 16.1/16.1 MB 6.6 MB/s eta 0:00:00


In [304]:
import mysql.connector
import pandas as pd

#Load cleaned Amazon and Ebay datasets
amazon_df = pd.read_csv("AmazonWebScrapingDataset_Cleaned.csv")
ebay_df = pd.read_csv("EbayWebScrapingDataset_Cleaned.csv")

#Connect to MySQL
conn = mysql.connector.connect(
    host= "localhost",
    user= "root",
    password= "Cognizant@121120",
    database= "PriceComparison"
)
cursor = conn.cursor()

#Insert Amazon Data
for _, row in amazon_df.iterrows():
    cursor.execute(
        "INSERT INTO AmazonData (product_title, price, product_link) VALUES (%s, %s, %s)",
        (row['Product Title'], row['Price'], row['Product Link'])
    )

#Insert Ebay Data
for _, row in ebay_df.iterrows():
    cursor.execute(
        "INSERT INTO EbayData (product_title, price, product_link) VALUES (%s, %s, %s)",
        (row['Product Title'], row['Price'], row['Product Link'])
    )

#Commit and Close Connection
conn.commit()
cursor.close()
conn.close()
print("Data successfully inserted into MySQL!")

Data successfully inserted into MySQL!
